# Ambiente configurado

In [ ]:
!pip install python_speech_features


  Preparing metadata (setup.py) ... done
  Created wheel for python_speech_features: filename=python_speech_features-0.6-py3-none-any.whl size=5870 sha256=fc4336cee0f3c6659b4669297e22ca214cf58d6dea5efcef3673bc16c72b6ef1
  Stored in directory: /root/.cache/pip/wheels/5a/9e/68/30bad9462b3926c29e315df16b562216d12bdc215f4d240294
Successfully built python_speech_features


In [ ]:
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
%pylab inline
import numpy as np
import os
import pandas as pd
import soundfile as sf
%config InlineBackend.figure_format = 'retina'
import tensorflow as tf;
import keras;
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, roc_curve
from python_speech_features import mfcc

Populating the interactive namespace from numpy and matplotlib


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Treino e teste

In [ ]:
%%time
from keras.callbacks import EarlyStopping
INPUT_DIR='/content/drive/MyDrive/TCC'
def build_lstm_model(input_shape):
    model = Sequential([
        LSTM(units=64, input_shape=input_shape, return_sequences=True),
        LSTM(units=32, input_shape=input_shape, return_sequences=False),# 'return_sequences' é False se você não está empilhando outra camada LSTM depois desta
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


def train_model(fold, base_dir):
    fold_path = os.path.join(base_dir, f'fold_{fold}_data.npz')

    # Carregar dados
    data = np.load(fold_path)
    X_train = data['X_train']
    y_train = data['y_train']
    X_test = data['X_test']
    y_test = data['y_test']

    # Construir o modelo
    model = build_lstm_model((24, 1))

    # Configurar retorno de chamada de parada antecipada
    early_stopping = EarlyStopping(monitor='val_loss', patience=75, restore_best_weights=True)

    # Treinar o modelo com parada antecipada
    history = model.fit(X_train, y_train, epochs=2500, batch_size=8, validation_data=(X_test, y_test), callbacks=[early_stopping], verbose=0)

    # Avaliar o modelo
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)


    # Fazer previsões
    y_pred_proba = model.predict(X_test).flatten()
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    print(f"Fold {fold}, roc_auc: {roc_auc}")
    print(f"Fold {fold}, Accuracy: {accuracy}")
    return fpr, tpr, roc_auc, accuracy



# Pasta com os k-folds
base_dir = os.path.join(INPUT_DIR, "Folds")

# Treinar o modelo e avaliar
results = [train_model(fold, base_dir) for fold in range(5)]
roc_auc_scores = [result[2] for result in results]
acc = [result[3] for result in results]

3/3 [==============================] - 1s 5ms/step
Fold 0, roc_auc: 0.6910331384015594
Fold 0, Accuracy: 0.6881720423698425
3/3 [==============================] - 1s 5ms/step
Fold 1, roc_auc: 0.7064950980392157
Fold 1, Accuracy: 0.782608687877655
3/3 [==============================] - 1s 5ms/step
Fold 2, roc_auc: 0.705528188286809
Fold 2, Accuracy: 0.72826087474823
3/3 [==============================] - 1s 5ms/step
Fold 3, roc_auc: 0.7458689458689459
Fold 3, Accuracy: 0.77173912525177


3/3 [==============================] - 1s 6ms/step
Fold 4, roc_auc: 0.6155223880597015
Fold 4, Accuracy: 0.72826087474823
CPU times: user 4min 3s, sys: 14 s, total: 4min 17s
Wall time: 4min 26s


In [ ]:
print("Mean ROC AUC:", np.mean(roc_auc_scores))
print("Mean ACC:", np.mean(acc))

Mean ROC AUC: 0.6919724019191967
Mean ACC: 0.7398316979408264


In [ ]:
print("ROC AUC Scores:", roc_auc_scores)
print("ACCScores:", acc)

ROC AUC Scores: [0.6244612068965517, 0.7383689107827038, 0.6785714285714286, 0.6328671328671329, 0.6962233169129721]
ACCScores: [0.7204301357269287, 0.77173912525177, 0.717391312122345, 0.75, 0.717391312122345]
